Before we get started, let's copy our copy of the relevant compiled tensorflow tools to the bin directory.
This is compiled for the ubuntu 22.04 machines running x86_64.
We should probably build a docker container to make these tools, but this one in particular takes 30 minutes to build on n2-standard-8.

In [31]:
tf_bin_path = "~/tensorflow/bazel-bin/"
data_bin_path = "../data/bin/linux/"
! mkdir -p {data_bin_path}
! cp -u {tf_bin_path}/tensorflow/tools/graph_transforms/transform_graph {data_bin_path}/transform_graph
! cp -u {tf_bin_path}/tensorflow/tools/graph_transforms/summarize_graph {data_bin_path}/summarize_graph
# also copy libtensorflow_framework.so.2 which is needed by both of the tools above
! cp -u {tf_bin_path}/tensorflow/libtensorflow_framework.so.2 {data_bin_path}/libtensorflow_framework.so.2
! gsutil rsync -r {data_bin_path} gs://birdclef-2023/data/bin/linux/

Building synchronization state...
Starting synchronization...
Copying file://../data/bin/linux/libtensorflow_framework.so.2 [Content-Type=application/octet-stream]...
- [1 files][ 38.3 MiB/ 38.3 MiB]                                                
Operation completed over 1 objects/38.3 MiB.                                     


In [32]:
# you might consider adding this directory to your PATH
tf_bin_path = "../data/bin/linux/"
model_path = "../vendor/BirdNET-Analyzer/checkpoints/V2.2/BirdNET_GLOBAL_3K_V2.2_Model/"
frozen_graph_v1 = "../data/models/birdnet-frozen-v1.pb"
opt_graph_v1 = "../data/models/birdnet-opt-v1.pb"
onnx_model_v1 = "../data/models/birdnet-onnx-v1.onnx"

! gsutil rsync -r gs://birdclef-2023/data/bin/linux/ {tf_bin_path}
! mkdir -p ../data/models

Building synchronization state...
Starting synchronization...


- https://github.com/tensorflow/tensorflow/tree/master/tensorflow/tools/graph_transforms#using-the-graph-transform-tool
- https://www.tensorflow.org/guide/saved_model
- https://www.tensorflow.org/guide/saved_model#specifying_signatures_during_export
- https://developer.arm.com/documentation/ecm0744361/ab/Determine-the-names-of-input-and-output-nodes

In [21]:
import tensorflow as tf
from tensorflow.python.framework.convert_to_constants import (
    convert_variables_to_constants_v2,
)

m = tf.saved_model.load(model_path)
frozen_func = convert_variables_to_constants_v2(m.signatures["basic"])
tf.io.write_graph(
    graph_or_graph_def=frozen_func.graph,
    logdir="./",
    name=frozen_graph_v1,
    as_text=False,
)

'./../data/models/birdnet-frozen-v1.pb'

In [33]:
! {tf_bin_path}/transform_graph \
    --in_graph={frozen_graph_v1} \
    --out_graph={opt_graph_v1} \
    --inputs='inputs:0' \
    --transforms=' \
        remove_nodes(op=Identity, op=CheckNumerics) \
        fold_old_batch_norms \
    '

In [34]:
! {tf_bin_path}/summarize_graph \
    --in_graph={opt_graph_v1} \
    --print_structure=true

Found 1 possible inputs: (name=inputs, type=float(1), shape=[?,144000]) 
No variables spotted.
Found 1 possible outputs: (name=StatefulPartitionedCall/model/CLASS_DENSE_1/BiasAdd, op=Add) 
Found 5488973 (5.49M) const parameters, 0 (0) variable parameters, and 246 control_edges
Op types used: 363 Const, 53 Conv2D, 49 Mul, 45 FusedBatchNormV3, 43 Sigmoid, 34 IdentityN, 17 AddV2, 15 Pack, 14 Reshape, 12 StridedSlice, 10 Mean, 10 Shape, 9 DepthwiseConv2dNative, 7 Sub, 4 FloorDiv, 4 ConcatV2, 4 Cast, 4 Range, 3 Relu, 3 RealDiv, 2 Pow, 2 Add, 1 RFFT, 1 SplitV, 1 Placeholder, 1 Transpose, 1 NoOp, 1 Min, 1 Maximum, 1 MaxPool, 1 Max, 1 MatMul, 1 GatherV2, 1 FloorMod, 1 Fill, 1 ExpandDims, 1 Exp, 1 Cos, 1 AvgPool
To use with tensorflow/tools/benchmark:benchmark_model try these arguments:
bazel run tensorflow/tools/benchmark:benchmark_model -- --graph=../data/models/birdnet-opt-v1.pb --show_flops --input_layer=inputs --input_layer_type=float --input_layer_shape=-1,144000 --output_layer=StatefulPa

In [24]:
! python -m tf2onnx.convert \
  --input {opt_graph_v1} \
  --inputs inputs:0 \
  --outputs unknown_242:0,unknown_244:0 \
  --output {onnx_model_v1}

/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
2023-02-26 05:37:39,160 - WARNING - From /home/anthony/birdclef-2023/venv/lib/python3.10/site-packages/tf2onnx/tf_loader.py:302: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to mi

In [35]:
import onnxruntime as rt
import numpy as np

test = np.random.randn(1, 144_000).astype(np.float32)

sess = rt.InferenceSession(onnx_model_v1)
y1 = sess.run(None, {"inputs:0": test})[0]

- https://leimao.github.io/blog/Save-Load-Inference-From-TF2-Frozen-Graph/

We re-use a bit of code found from a random blog post.
We basically want to use the frozen graph to do inference, because it has the same node names as the output onnx graph.
We could use the saved_model directly, but we would need to make sure to get the correct embedding layer from it.

In [36]:
def wrap_frozen_graph(graph_def, inputs, outputs):
    def _imports_graph_def():
        tf.compat.v1.import_graph_def(graph_def, name="")

    wrapped_import = tf.compat.v1.wrap_function(_imports_graph_def, [])
    import_graph = wrapped_import.graph

    return wrapped_import.prune(
        tf.nest.map_structure(import_graph.as_graph_element, inputs),
        tf.nest.map_structure(import_graph.as_graph_element, outputs),
    )


# load optimized graph
with tf.io.gfile.GFile(opt_graph_v1, "rb") as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

opt_tf_model = wrap_frozen_graph(graph_def, inputs="inputs:0", outputs="unknown_242:0")
tensor = tf.convert_to_tensor(test)
y2 = opt_tf_model(tensor).numpy()

In [37]:
y1.shape, y2.shape

((320,), (320,))

In [38]:
np.allclose(y1, y2)

True

In [39]:
for i in range(100):
    test = np.random.randn(1, 144_000).astype(np.float32)
    y1 = sess.run(None, {"inputs:0": test})[0]
    y2 = opt_tf_model(tensor).numpy()
    assert np.allclose(y1, y2)
print("ok")

ok
